---
# Docker

> Fast AI in Docker Container
- key1: value1
- key2: value2
- categories: [c1, c2]
- skip_showdoc: true
- skip_exec: true

format:
  html:
    theme:
      light: flatly
      dark: darkly
    mermaid:
      theme: forest


---

## Definitions

- Dockerfile: Blueprint for building a docker image
 
- compose.yml: Builds and runs multiple docker images
 
- requirements.txt: pip install libraries list 
- Image: Template for running a docker containers
- Container: Running process

## Docker File


`FILE _image_` - base image ex. ubuntu, node:12

`WORKDIR _filename_`- like when you cd into a directory

**_every step from now is considered layes_**

`COPY _filename_ _location_`- ex. COPY package*.json ./
    
`RUN _npm install_` - exec commands - shell form

`COPY . .` - copy all the file in directory into the container

`ENV _PORT=8080_` - 

`EXPOSE _port_` - exposed port ex. 8080

`CMD ["npm" "start"]` - only one of these in the file. exec form



### Example: pytorch image

```Dockerfile
# Base image with PyTorch and GPU support
FROM pytorch/pytorch:latest

# Install additional packages
RUN apt-get update && \
    apt-get install -y sudo software-properties-common

RUN add-apt-repository -y ppa:bashtop-monitor/bashtop && \
    add-apt-repository -y ppa:flexiondotorg/nvtop && \
    apt-get update

RUN apt-get install -y bashtop nvtop python3-pip \
    vim poppler-utils git openssh-client && \
    apt-get clean

# Add user 'ben'
RUN useradd -m -s /bin/bash ben && \
    echo "ben ALL=(ALL) NOPASSWD:ALL" >> /etc/sudoers

# Set working directory
WORKDIR /home/ben

# Switch to user ben
USER ben

# Add /home/ben/.local/bin to the PATH
ENV PATH="/home/ben/.local/bin:${PATH}"

# Copy requirements file
COPY requirements.txt .

# Install required Python packages
RUN pip install --no-cache-dir -r requirements.txt

RUN git config --global user.email <email> \
    && git config --global user.name "main_docker"

# COPY /home/ben/'.jupyter' '.jupyter'
ADD /apputils-extension /home/ben/.jupyter/lab/user-settings/@jupyterlab/apputils-extension

# Expose Jupyter notebook port
EXPOSE 8888

# Command to start JupyterLab with dark theme
CMD ["jupyter", "lab", "--ip=0.0.0.0", "--port=8888", "--no-browser", "--NotebookApp.theme='JupyterLab Dark'", "--NotebookApp.token='test'"]
```

### .dockerignore 

> just like a git ignore file

### Building Docker image

```sh
docker build -t _name_ _path_
```
ex. 
```sh
docker build -t test:taggedv0.1
```
-t : tag

### View Docker Images

```sh
docker images
```

### Delete Image

```sh
docker rmi _Image_
```

### Running Docker image as Container

```sh
docke run -p 5000:8080 _name_

```

```sh
docker run -p 8888:8888 -td --gpus=all --name testcontainer test

```

### View Dockers Containers Running

```sh
docker ps -a
```

### Deleting Docker Containers

```sh

docker rmi _image_
```

### Stop Docker Running

```sh
docker stop $(docker ps -aq)
```

## Volumes 

> Folder which multiple containers can access

```sh
docker volume create _name_

```

### Using volumes in container

```sh
docker run --mount source=_volumename_, target = _location_

```

## Docker Debugging

- Inspect logs in docker desktop

- docker exec

## TIPs

- each docker should only do one task

- Done through docker-compose.yml

## Compose.yml

### Example

> this [GPU](https://docs.docker.com/compose/gpu-support/)

> [nvidia Docs](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/1.14.5/install-guide.html)
```yml
services:
  torch:
    container_name: main_docker
    
    build:
      context: .

    volumes:
      - /home/ben/BENEDICT_Only:/home/ben/BENEDICT_Only
      
    ports:
      - "8888:8888"
    
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: 1
              capabilities: [gpu]

```

```yml
version: '3'
services:
    web: 
        build: .
        ports:
            - "8080:8080"
    db: 
        image: "mysql"
        environment:
            MYSQL_ROOT_PASSWORD: password
        volumes:
            - db-data:/foo
            
volumes:
    db-data:
            

```

### Running Docker Compose

```sh
docker-compose up

```

### Stopping Docker Compose

```sh
docker-compose down
```

## requirements.txt

### Example

```txt
jupyterlab
fastai
fastbook
timm
pdf2image
pandas
nbdev
nbdevAuto
```